In [ ]:
# %% [markdown]
# # Análise de Redes Complexas para Nódulos Polimetálicos
# 
# Este notebook demonstra como usar o módulo LETE para construir e analisar redes complexas 
# que representam relações entre locais de mineração, portos e fatores econômicos.

# %%
# Importar bibliotecas
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

# Adicionar o diretório src ao path
sys.path.append(os.path.abspath('../'))

# Importar o módulo LETE
from src.network.lete import LETECalculator

# Configurar visualizações
plt.style.use('seaborn-whitegrid')
sns.set(style="whitegrid", font_scale=1.2)

# %% [markdown]
# ## 1. Dados de Exemplo para Análise de Rede
# 
# Para esta demonstração, criaremos um conjunto de dados simulado representando:
# - Densidade de nódulos em diferentes locais
# - Preços de metais ao longo do tempo
# - Custos de extração para diferentes sites
# - Distâncias a portos e capacidades de processamento

# %%
# Gerar dados simulados
np.random.seed(42)

# Período de tempo (meses)
n_periods = 60
dates = pd.date_range(start='2018-01-01', periods=n_periods, freq='M')

# Sites de mineração simulados
sites = ['Rio_Grande_Rise_A', 'Rio_Grande_Rise_B', 'Mid_Atlantic_Ridge', 
         'Bermuda_Rise', 'Azores_Plateau']

# Portos simulados
ports = ['Port_Brazil', 'Port_US_East', 'Port_West_Africa', 'Port_Europe']

# Metais presentes nos nódulos
metals = ['Nickel', 'Copper', 'Cobalt', 'Manganese']

# Gerar preços de metais com tendências e sazonalidades
prices_df = pd.DataFrame(index=dates)
base_prices = {'Nickel': 15000, 'Copper': 8000, 'Cobalt': 30000, 'Manganese': 1700}

for metal in metals:
    # Tendência linear + componente sazonal + componente auto-regressivo + ruído
    trend = np.linspace(0, 0.3, n_periods)  # Tendência ascendente
    seasonality = 0.1 * np.sin(np.linspace(0, 6*np.pi, n_periods))  # Ciclo sazonal
    
    # Componente auto-regressivo
    ar_component = np.zeros(n_periods)
    ar_component[0] = np.random.normal(0, 0.02)
    for i in range(1, n_periods):
        ar_component[i] = 0.8 * ar_component[i-1] + np.random.normal(0, 0.02)
    
    # Ruído aleatório
    noise = np.random.normal(0, 0.05, n_periods)
    
    # Preço final
    price = base_prices[metal] * (1 + trend + seasonality + ar_component + noise)
    prices_df[metal] = price

# Gerar densidades de nódulos com dependências causais
site_data = pd.DataFrame(index=dates)

for site in sites:
    # Densidade base varia por site
    base_density = {
        'Rio_Grande_Rise_A': 12.0,
        'Rio_Grande_Rise_B': 10.5,
        'Mid_Atlantic_Ridge': 8.0,
        'Bermuda_Rise': 7.0,
        'Azores_Plateau': 6.5
    }[site]
    
    # A densidade é afetada pelo preço do níquel com defasagem
    nickel_effect = 0.1 * (prices_df['Nickel'].shift(2) - prices_df['Nickel'].mean()) / prices_df['Nickel'].std()
    
    # Também é afetada pelo preço do cobalto com defasagem
    cobalt_effect = 0.05 * (prices_df['Cobalt'].shift(1) - prices_df['Cobalt'].mean()) / prices_df['Cobalt'].std()
    
    # E tem um componente aleatório
    random_effect = np.random.normal(0, 0.2, n_periods)
    
    # Densidade final (kg/m²)
    density = base_density + nickel_effect + cobalt_effect + random_effect
    site_data[f"{site}_density"] = density.clip(lower=0)  # Não permitir densidade negativa

# Gerar custos de extração (USD/ton)
for site in sites:
    # Custo base varia por site
    base_cost = {
        'Rio_Grande_Rise_A': 300,
        'Rio_Grande_Rise_B': 320,
        'Mid_Atlantic_Ridge': 350,
        'Bermuda_Rise': 380,
        'Azores_Plateau': 400
    }[site]
    
    # Custos são afetados pelo preço do petróleo (simulado)
    oil_price = 80 + 20 * np.sin(np.linspace(0, 4*np.pi, n_periods)) + np.random.normal(0, 5, n_periods)
    oil_effect = 0.3 * (oil_price - oil_price.mean()) / oil_price.std()
    
    # Também são afetados pela densidade (maior densidade = menor custo por tonelada)
    density_effect = -0.2 * (site_data[f"{site}_density"] - site_data[f"{site}_density"].mean()) / site_data[f"{site}_density"].std()
    
    # Custo final
    cost = base_cost * (1 + oil_effect + density_effect)
    site_data[f"{site}_cost"] = cost

# %% [markdown]
# ## 2. Visualizar os Dados Simulados

# %%
# Visualizar preços de metais
plt.figure(figsize=(12, 6))
for metal in metals:
    plt.plot(prices_df.index, prices_df[metal], label=metal)
plt.title('Preços Simulados de Metais (USD/ton)')
plt.xlabel('Data')
plt.ylabel('Preço (USD/ton)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# %%
# Visualizar densidades de nódulos
plt.figure(figsize=(12, 6))
for site in sites:
    plt.plot(site_data.index, site_data[f"{site}_density"], label=site)
plt.title('Densidades Simuladas de Nódulos (kg/m²)')
plt.xlabel('Data')
plt.ylabel('Densidade (kg/m²)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# %%
# Visualizar custos de extração
plt.figure(figsize=(12, 6))
for site in sites:
    plt.plot(site_data.index, site_data[f"{site}_cost"], label=site)
plt.title('Custos Simulados de Extração (USD/ton)')
plt.xlabel('Data')
plt.ylabel('Custo (USD/ton)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 3. Análise LETE das Séries Temporais
# 
# Vamos usar o módulo LETE para analisar a transferência de informação entre preços,
# densidades e custos.

# %%
# Preparar dados para análise LETE
# Vamos usar os últimos 24 meses para a análise
analysis_period = 24
recent_data = pd.DataFrame()

# Adicionar preços dos metais
for metal in metals:
    recent_data[metal] = prices_df[metal].iloc[-analysis_period:].values

# Adicionar densidades dos sites
for site in sites:
    recent_data[f"{site}_density"] = site_data[f"{site}_density"].iloc[-analysis_period:].values

# Adicionar custos dos sites
for site in sites:
    recent_data[f"{site}_cost"] = site_data[f"{site}_cost"].iloc[-analysis_period:].values

# %%
# Calcular matriz LETE
lete_calc = LETECalculator(k=1, l=1, bins=5, shuffle_iterations=50)
lete_matrix = lete_calc.calculate_lete_matrix(recent_data)

# Visualizar matriz LETE completa
plt.figure(figsize=(14, 12))
sns.heatmap(lete_matrix, annot=False, cmap='viridis', 
            xticklabels=lete_matrix.columns, yticklabels=lete_matrix.columns)
plt.title('Matriz LETE Completa')
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 4. Construção da Rede Complexa
# 
# Agora vamos usar os valores LETE para construir uma rede complexa que mostra
# as relações causais entre as diferentes variáveis.

# %%
# Criar rede usando networkx
G = nx.DiGraph()

# Adicionar nós
for col in lete_matrix.columns:
    # Determinar o tipo de nó para coloração
    if col in metals:
        node_type = 'metal'
    elif 'density' in col:
        node_type = 'density'
    elif 'cost' in col:
        node_type = 'cost'
    else:
        node_type = 'other'
        
    G.add_node(col, type=node_type)

# Adicionar arestas baseadas em LETE
# Usamos um threshold para incluir apenas relações significativas
threshold = 0.1
for i, source in enumerate(lete_matrix.columns):
    for j, target in enumerate(lete_matrix.columns):
        if i != j and lete_matrix.iloc[i, j] > threshold:
            G.add_edge(source, target, weight=lete_matrix.iloc[i, j])

# %%
# Visualizar a rede complexa
plt.figure(figsize=(16, 12))

# Definir posição dos nós
pos = nx.spring_layout(G, k=0.3, seed=42)

# Definir cores por tipo de nó
color_map = {
    'metal': 'gold',
    'density': 'limegreen',
    'cost': 'tomato',
    'other': 'skyblue'
}
node_colors = [color_map[G.nodes[node]['type']] for node in G.nodes()]

# Obter pesos das arestas para definir largura
edge_weights = [G[u][v]['weight'] * 3 for u, v in G.edges()]

# Desenhar nós
nx.draw_networkx_nodes(G, pos, node_size=700, node_color=node_colors, alpha=0.8)

# Desenhar arestas com largura proporcional ao peso
nx.draw_networkx_edges(G, pos, width=edge_weights, alpha=0.7, 
                      edge_color='gray', arrows=True, arrowsize=15,
                      connectionstyle='arc3,rad=0.1')

# Adicionar rótulos
nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')

# Adicionar legenda para os tipos de nós
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, 
                              markersize=10, label=node_type)
                  for node_type, color in color_map.items()]
plt.legend(handles=legend_elements, loc='upper right')

# Adicionar barra de cores para os pesos das arestas
sm = ScalarMappable(norm=Normalize(vmin=threshold, vmax=max(edge_weights)/3), 
                   cmap=plt.cm.gray)
sm.set_array([])
cbar = plt.colorbar(sm, label='LETE Value')

plt.title('Rede Complexa de Transferência de Informação para Nódulos Polimetálicos')
plt.axis('off')
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 5. Análise de Centralidade da Rede
# 
# Vamos calcular métricas de centralidade para identificar os nós mais influentes
# na rede.

# %%
# Calcular métricas de centralidade
centrality_measures = {}

# Centralidade de entrada (quem recebe mais informação)
centrality_measures['in_degree'] = dict(G.in_degree(weight='weight'))

# Centralidade de saída (quem envia mais informação)
centrality_measures['out_degree'] = dict(G.out_degree(weight='weight'))

# Centralidade de intermediação (quem está no caminho entre outros nós)
centrality_measures['betweenness'] = nx.betweenness_centrality(G, weight='weight')

# Centralidade de autovetor (importância baseada na importância dos vizinhos)
centrality_measures['eigenvector'] = nx.eigenvector_centrality(G, weight='weight')

# %%
# Criar DataFrame com todas as medidas de centralidade
centrality_df = pd.DataFrame(centrality_measures)

# Ordenar pelo grau de saída (quem mais influencia outros nós)
centrality_df = centrality_df.sort_values('out_degree', ascending=False)

# Mostrar resultados
print("Medidas de Centralidade da Rede:")
centrality_df

# %%
# Visualizar medidas de centralidade
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
measures = list(centrality_measures.keys())

for i, (measure, ax) in enumerate(zip(measures, axes.flatten())):
    # Ordenar os valores
    values = {k: centrality_measures[measure][k] for k in 
              sorted(centrality_measures[measure], key=centrality_measures[measure].get, reverse=True)}
    
    # Definir cores baseadas no tipo de nó
    colors = [color_map[G.nodes[node]['type']] for node in values.keys()]
    
    # Criar gráfico de barras
    bars = ax.bar(values.keys(), values.values(), color=colors)
    
    # Adicionar rótulos
    ax.set_title(f'Centralidade: {measure}')
    ax.set_ylabel('Valor')
    ax.tick_params(axis='x', rotation=90)
    
    # Adicionar grid
    ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# %% [markdown]
# ## 6. Implicações para a Viabilidade Econômica
# 
# Baseado na análise LETE e na rede complexa, podemos extrair insights sobre a viabilidade
# econômica da exploração de nódulos polimetálicos no Atlântico.

# %%
# Identificar os nós mais influentes (maiores valores de saída)
top_influencers = centrality_df.sort_values('out_degree', ascending=False).head(5)
print("Nós mais influentes (maiores valores de saída):")
top_influencers

# %%
# Identificar os nós mais influenciados (maiores valores de entrada)
top_influenced = centrality_df.sort_values('in_degree', ascending=False).head(5)
print("Nós mais influenciados (maiores valores de entrada):")
top_influenced

# %% [markdown]
# ## 7. Conclusões e Próximos Passos
# 
# Neste notebook, demonstramos como usar o LETE para analisar relações causais entre 
# variáveis relevantes para a exploração de nódulos polimetálicos no Atlântico.
# 
# Os principais insights são:
# 
# 1. Os preços dos metais, especialmente [identificar metais mais influentes], têm forte 
#    influência nas densidades e custos de extração.
# 
# 2. Certos sites de mineração, como [identificar sites relevantes], mostram maior sensibilidade 
#    às variações nos preços dos metais.
# 
# 3. A análise de centralidade revela que [discutir resultados relevantes].
# 
# Próximos passos:
# 
# 1. Coletar dados reais de preços de metais, densidades de nódulos e custos operacionais
# 
# 2. Aplicar a metodologia LETE aos dados reais
# 
# 3. Desenvolver um modelo de otimização de portfólio baseado em LETE para selecionar 
#    os melhores locais de mineração